In [1]:
RA = 59.53159583
# RA = 3.96877306
DEC = -47.92468056
RADIUS_ARCSEC = 5.0

In [2]:
%pip install lsdb 'dask<2025'

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Relevant Directories
#ls /sdf/scratch/users/n/ncaplar/HATS

comcam_obj = "/sdf/data/rubin/shared/lsdb_commissioning/Object_50"
comcam_src = "/sdf/data/rubin/shared/lsdb_commissioning/forcedSource_50"
comcam_diasrc = "/sdf/data/rubin/shared/lsdb_commissioning/diaSource_50"
comcam_srcmjd = "/sdf/data/rubin/shared/lsdb_commissioning/forcedSource_50_with_mjd"

In [4]:
from dask.distributed import Client

# Start with a small client
client = Client(n_workers=24, memory_limit="8GB", threads_per_worker=1)
client

/sdf/home/k/kostya/rubin-user/linccf/.venv/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39177 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39177/status,
Dashboard: http://127.0.0.1:39177/status,Workers: 24
Total threads: 24,Total memory: 178.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42407,Workers: 24
Dashboard: http://127.0.0.1:39177/status,Total threads: 24
Started: Just now,Total memory: 178.81 GiB
Comm: tcp://127.0.0.1:32859,Total threads: 1
Dashboard: http://127.0.0.1:39527/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:39809,


In [5]:
# Load the Forced Source + MJD Table
import nested_dask as nd
from lsdb import ConeSearch, read_hats

src_mjd_ddf = nd.read_parquet("/sdf/data/rubin/shared/lsdb_commissioning/forcedSource_50_with_mjd", calculate_divisions=True)
src_nested = nd.NestedFrame.from_flat(
    src_mjd_ddf,
#   base_columns=["coord_ra", "coord_dec"],
    base_columns=[],
    nested_columns=["band", "psfFlux", "psfFluxErr", "midpointMjdTai"],
    name="lc",
)

# Grab columns from the Object Catalog
from lsdb import read_hats
obj_ddf = read_hats(
    comcam_obj, 
    columns=["objectId", "coord_ra", "coord_dec", "g_psfFlux", "r_psfFlux"],
    search_filter=ConeSearch(RA, DEC, radius_arcsec=RADIUS_ARCSEC),
)._ddf.set_index("objectId")

# LSDB joins with non-catalogs don't really work, so cheating by using native Dask/Nested-Dask
src_nested = src_nested.join(obj_ddf).dropna(subset=["g_psfFlux", "r_psfFlux"])

src_nested

,lc,coord_ra,coord_dec,g_psfFlux,r_psfFlux
npartitions=310,,,,,
199279885464633345,"nested<band: [large_string], psfFlux: [double], psfFluxErr: [double], midpointMjdTai: [double]>",double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
199548166301815239,...,...,...,...,...
...,...,...,...,...,...
4708143974508930760,...,...,...,...,...
4708236333485657710,...,...,...,...,...


In [6]:
src_nested.compute()

,lc,coord_ra,coord_dec,g_psfFlux,r_psfFlux
objectId,,,,,
